In [1]:
import pandas as pd
from urllib.parse import urlparse

In [2]:
# Load and get shape to same format as export in deduplication notebook
path = '/content/Test2020-11-09 03_02_06.620162+00_00.csv'
df = pd.read_csv(path, sep='|')
df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')
df['lat'] = df['lat'].index.astype(float)
df['long'] = df['long'].index.astype(float)
print(df.columns)
# Drop all unneccessary columns
df.drop(labels=['date', 'id', 'city', 'state', 'lat', 'long', 'title',
       'description', 'tags', 'verbalization', 'empty_hand_soft',
       'empty_hand_hard', 'less_lethal_methods', 'lethal_force',
       'uncategorized'], axis=1, inplace=True)

col = df.links

def stringtolinks(col):
    """
    Takes a column containing multiple urls as one string and parses urls
    """ 
    l, words, chars = [], [], []

    for row in range(len(col)):
        for char in col[row]:
            if (ord(char) != ord("'")) and (ord(char) != ord("[")) and (ord(char) != ord("]")) and (char != " "):
                    if char != ",":
                        chars.append(char)
                    else:
                        words.append("".join(chars))
                        chars = []

        words.append("".join(chars))  
        l.append(words)
        chars = []
        words =[]
    return l

df['links'] = stringtolinks(col)


def getSourcesandPaths(col):
    """
    A function to iterate through urls and split the source and path.
    """
    source, path = [], []

    for i, row  in enumerate(col):
        source_p, path_p = [], []

        for item in row:

            url = item
            o = urlparse(url)
            source_p.append(o.netloc)
            
            # source_p
            path_p.append(o.path)

        source.append(set(source_p))
        path.append(path_p)
    # print(len(source), len(paths))
    return source, path

df['source'], df['paths'] = getSourcesandPaths(col)


col = df.paths
def splitusers(col):
    """
    A function users to parse path prefix where a identifiable username
    can be found.
    """
    users = []
    user = set()
    chars = ""
    counter = 0

    for row in col:
        for item in row:
            for char in item:
                
                # if char == "/" and counter != 2:
                if (char == "/") and (counter < 2):
                    counter += 1 
                else:
                    chars = chars + char

                if counter == 2:
                    break
                    continue

            user.add(chars)
            chars = ""
            counter = 0

        users.append(user)
        user = set()
        chars = ""
        counter = 0

    return users


df['users'] = splitusers(col)
df.drop(labels=['paths'], axis=1, inplace=True)
df

Index(['date', 'links', 'id', 'city', 'state', 'lat', 'long', 'title',
       'description', 'tags', 'verbalization', 'empty_hand_soft',
       'empty_hand_hard', 'less_lethal_methods', 'lethal_force',
       'uncategorized'],
      dtype='object')


,links,source,users
0,[https://www.facebook.com/damicedsota.thespiri...,{www.facebook.com},{damicedsota.thespiritflow}
1,[https://www.facebook.com/1462345700/posts/102...,{www.facebook.com},{1462345700}
2,"[https://youtu.be/XAa5xb6JitI?t=5982, https://...","{youtu.be, gfycat.com}","{XAa5xb6JitI, distinctsecretgrasshopper-minnea..."
3,[https://www.denverpost.com/2020/05/29/denver-...,"{www.nytimes.com, www.denverpost.com}",{2020}
4,[https://twitter.com/ShimonPro/status/12661365...,{twitter.com},{ShimonPro}
...,...,...,...
1244,[https://twitter.com/misstessowen/status/13242...,{twitter.com},{misstessowen}
1245,[https://twitter.com/desertborder/status/13241...,"{twitter.com, www.latimes.com}","{california, ACatWithNews, SamBraslow, VPS_Rep..."
1246,[https://twitter.com/jangelooff/status/1324167...,{twitter.com},{jangelooff}
1247,[https://twitter.com/Cascadianphotog/status/13...,{twitter.com},"{ByMikeBaker, hungrybowtie, Cascadianphotog}"


In [3]:
url = df.links[0][0]  # parse a single url
o = urlparse(url)
o.netloc, o.path

('www.facebook.com',
 '/damicedsota.thespiritflow/videos/10216865788705633/UzpfSTEwMDAxMTAzODkyNjEwMzpWSzoyNjczNDU4ODUyOTMzODE2/')

In [4]:
df.iloc[0]

links     [https://www.facebook.com/damicedsota.thespiri...
source                                   {www.facebook.com}
users                           {damicedsota.thespiritflow}
Name: 0, dtype: object

In [5]:
df.users.iloc[0]

{'damicedsota.thespiritflow'}